In [16]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.optimize as opt

In [621]:
def stationary_normal_bandit(k, sigma=.5, q=None):
    q = q.copy() if q is not None else \
        np.random.uniform(low=0.0, high=1.0, size=(k,))

    def env(a):
        return q, q[a], np.random.normal(loc=q[a], scale=sigma)

    return env

In [645]:
def epsilon_greedy_agent(k, epsilon):
    Q = np.zeros((k,))
    N = np.zeros((k,))
    t = 0

    def step():
        nonlocal t

        if np.random.random() < epsilon:
            A = np.random.choice(np.arange(k))
        else:
            A = np.argmax(Q)

        R = yield A

        N[A] += 1
        Q[A] += (1 / N[A]) * (R - Q[A])
        t += 1

    return step

In [21]:
# def constant_step_agent(epsilon, alpha, k):
#     Q = np.zeros((k,))
#     t = 1

#     def step():
#         if np.random.random() < (epsilon / (1 + np.log(t))):
#             A = np.random.choice(np.arange(k))
#         else:
#             A = np.argmax(Q)

#         R = yield A

#         Q[A] += alpha * (R - Q[A])
#         t += 1

#     return step

In [22]:
def ucb1_agent(k):
    t = 1
    Q = np.ones((k,))
    N = np.ones((k,))

    def step():
        nonlocal t

        A = np.argmax(Q + np.sqrt((2 * np.log(t)) / N))

        R = yield A

        t += 1
        N[A] += 1
        Q[A] += (1 / N[A]) * (R - Q[A])

    return step

In [23]:
def softmax_agent(k, tau):
    Q = np.ones((k,))
    N = np.ones((k,))

    def step():
        p = np.exp(Q / tau) / np.exp(Q / tau).sum()
        
        A = np.random.choice(np.arange(k), p=p)

        R = yield A

        N[A] += 1
        Q[A] += (1 / N[A]) * (R - Q[A])

    return step

In [24]:
def rc_agent(k, alpha, beta):
    R_m = 0
    pi = np.zeros((k,))
    
    def step():
        nonlocal R_m, pi
        
        p = np.exp(pi) / np.exp(pi).sum()
        
        A = np.random.choice(np.arange(k), p=p)

        R = yield A
        pi[A] += beta * (R - R_m)
        R_m *= (1 - alpha)
        R_m += alpha * R

    return step

In [617]:
def epsilon_extinct_agent(k, epsilon):
    N = np.zeros((k,))
    Q = np.ones((k,))
    t = 0

    def step():
        nonlocal t

        p = 1 / (1 + epsilon * t)

        if np.random.random() < p:
            A = np.random.choice(np.arange(k))
        else:
            A = np.argmax(Q)

        R = yield A

        N[A] += 1
        Q[A] += (1 / N[A]) * (R - Q[A])
        t += 1

    return step

In [390]:
def run(bandit, agent, bandit_parameters, agent_parameters, epochs, samples):
    regret = np.zeros((samples, epochs), dtype=np.float32)
    optimality = np.zeros((samples, epochs), dtype=np.float32)
    total = np.tile(np.arange(1, epochs + 1, dtype=np.int32), (samples, 1))

    for sample in range(samples):
        env = bandit(*bandit_parameters)
        step = agent(*agent_parameters)

        for epoch in range(epochs):
            generator = step()
            A = next(generator)
            q, mu, R = env(A)

            try:
                generator.send(R)
            except StopIteration:
                pass

            regret[sample, epoch] = np.max(q) - mu
            optimality[sample, epoch] = float(A == np.argmax(q))
    
    return (
        regret.cumsum(axis=1).mean(axis=0),
        (optimality.cumsum(axis=1) / total).mean(axis=0)
    )

In [629]:
def evaluate(
    bandit,
    agent,
    parameters,
    epochs=2000,
    samples=1
):
    return [
        run(
            bandit,
            agent,
            bandit_parameters,
            agent_parameters,
            epochs,
            samples
        )
        for bandit_parameters, agent_parameters in parameters
    ]

In [338]:
def plot(results, labels=None):
    if labels is None:
        labels = [None] * len(results)
    
    plt.style.use("seaborn-v0_8-pastel")

    fig, subplots = plt.subplots(2, 1, sharex=True)
    regret_subplot, optimality_subplot = subplots
    
    for (regret, optimality), label in zip(results, labels):
        x = np.arange(regret.shape[0])
        regret_subplot.plot(regret, label=label)
        optimality_subplot.plot(optimality, label=label)
    
    regret_subplot.set_ylabel("Regret")
    regret_subplot.legend()

    optimality_subplot.set_ylabel("Optimality")
    optimality_subplot.set_xlabel("Step")
    optimality_subplot.legend()

In [703]:
def minimize_extinction_epsilon(bandit, k, sigma):
    f = lambda epsilon: run(
        bandit,
        epsilon_extinct_agent,
        [k, sigma],
        [k, epsilon],
        1000,
        100
    )[0][-1]

    result = opt.minimize(f, [0.05], bounds=[(0.001, 1)], method='Powell')

    if not result.success:
        raise RuntimeError(result.message)

    return result.x[0]

In [704]:
def extinction_epsilon_variance_correlation(bandit, k, sigmas=None):
    if sigmas is None:
        sigmas = np.linspace(0.01, 1, 20)

    epsilons = np.zeros((sigmas.shape[0],))

    for index, sigma in enumerate(sigmas):
        epsilons[index] = minimize_extinction_epsilon(bandit, k, sigma)

    return sigmas, epsilons

In [705]:
sigmas, epsilons = extinction_epsilon_variance_correlation(stationary_normal_bandit, 10)

KeyboardInterrupt: 

In [ ]:
plt.plot(sigmas, epsilons)

In [ ]:
epochs = 100
samples = 100
r1, _ = run(stationary_normal_bandit, epsilon_greedy_agent, (10,), (10, 0.01), epochs, samples)
r2, _ = run(stationary_normal_bandit, epsilon_extinct_agent, (10,), (10, 0.5), epochs, samples)
plt.plot(r1)
plt.plot(r2)